# AML Use Case

In [ ]:
!pip install alibi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 523.0/523.0 kB 7.7 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 16.5 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached blis-0.7.9-cp38-cp38-linux_ppc64le.whl
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 3.0 MB/s eta 0:00:00
  Using cached murmurhash-1.0.9-cp38-cp38-linux_ppc64le.whl
  Using cached catalogue-2.0.8-py3-none-any.whl (17 kB)
  Using cached wasabi-1.1.1-py3-none-any.whl (27 kB)
  Using cached srsly-2.4.6-cp38-cp38-linux_ppc64le.whl
  Using cached thinc-8.1.8-cp38-cp38-linux_ppc64le.whl
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.6/181.6 kB 10.0 MB/s eta 0:00:00
  Using cached cymem-2.0.7-cp38-cp38-linux_ppc64le.whl
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 2.6 MB/s eta 0:00:00
  Using cached

In [6]:
## data from https://github.com/IBM/AML-Data
## !wget https://ibm.box.com/shared/static/gu098ispc0by71ubfm1px111n807xk53.csv

## Import Data From CSV

In [13]:
from datetime import datetime

dtypes = {
    'From Bank': str,
    'Account': str,
    'To Bank': str,
    'Account.1': str,
    'Amount Received': str,
    'Receiving Currency': str,
    'Amount Paid': str,
    'Payment Currency': str,
    'Payment Format': str,
    'Is Laundering': int,
}

In [17]:
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder 

data = pd.DataFrame()
oenc = OrdinalEncoder()

for chunk in pd.read_csv('ibm-aml-data-2.csv', chunksize=100_000, nrows=1_000_000, dtype=dtypes, parse_dates=['Timestamp'], index_col=0):
    data = pd.concat([data, chunk])

data.replace('(null)', pd.NA, inplace=True)
data.dropna(inplace=True)
data.sort_values('Timestamp', inplace=True)
data['Amount Paid'] = data['Amount Paid'].astype(float)
data['Amount Received'] = data['Amount Received'].astype(float)
data = data.set_index('Timestamp')
data.iloc[:, [0,1,2,3,5,7,8]] = oenc.fit_transform(data.iloc[:, [0,1,2,3,5,7,8]])
cols = list(data.columns)
cols.pop(cols.index('Is Laundering'))
data = data[['Is Laundering']+cols]
data.head()
len(data)

1000000

In [18]:
data.head()

,Is Laundering,From Bank,Account,To Bank,Account.1,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format
Timestamp,,,,,,,,,,
2019-01-01,0,10780.0,187014.0,10552.0,187940.0,164.760000,7.0,164.760000,7.0,4.0
2019-01-01,0,35786.0,588375.0,35845.0,616655.0,0.011102,1.0,0.011102,1.0,4.0
2019-01-01,0,8376.0,406686.0,254.0,68406.0,9.180000,12.0,9.180000,12.0,4.0
2019-01-01,0,686.0,406687.0,20959.0,377791.0,48.290000,12.0,48.290000,12.0,4.0
2019-01-01,0,4019.0,396610.0,6098.0,345197.0,8.740000,4.0,8.740000,4.0,4.0


In [19]:
from pickle import dump, load
dump(oenc, open('ordinal_encoder.pkl', 'wb'))
scaler = load(open('ordinal_encoder.pkl', 'rb'))

## Create Graph From Transactions

In [20]:
# inspired by:
# https://www.emerald.com/insight/content/doi/10.1108/JMLC-07-2019-0055/full/pdf?title=detecting-money-laundering-transactions-with-machine-learning

In [21]:
import networkx as nx

source = data['From Bank']+data['Account']
target = data['To Bank']+data['Account.1']

trans_graph = nx.DiGraph()
trans_graph.add_edges_from(list(zip(source, target)))

print('#Accounts:', len(trans_graph.nodes))
print('#Transactions:', sum(data.duplicated(subset=['Account', 'Account.1']))+len(trans_graph.edges))
print('#Connections:', len(trans_graph.edges))

#Accounts: 565715
#Transactions: 998673
#Connections: 944231


### Calculate Graph Features

In [22]:
cc = nx.clustering(trans_graph, weight=None)
ids = trans_graph.in_degree()
ods = trans_graph.out_degree()
dc = nx.degree_centrality(trans_graph)

In [23]:
data['Clustering Coeff'] = source.map(cc) + target.map(cc)
data['In Degree'] = source.map(ids) + target.map(ids)
data['Out Degree'] = source.map(ods) + target.map(ods)
data['Degree Centrality'] = source.map(dc) + target.map(dc)

In [24]:
from scipy.stats import zscore

transaction_history = (source + target).value_counts().to_dict()
data['No. of Past Transactions'] = (source+target).map(transaction_history)
data['Z-Score Diff'] = zscore(data['Amount Paid'])
data.head()

,Is Laundering,From Bank,Account,To Bank,Account.1,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,Clustering Coeff,In Degree,Out Degree,Degree Centrality,No. of Past Transactions,Z-Score Diff
Timestamp,,,,,,,,,,,,,,,,
2019-01-01,0,10780.0,187014.0,10552.0,187940.0,164.760000,7.0,164.760000,7.0,4.0,0.0,3,12,0.000027,2,-0.017984
2019-01-01,0,35786.0,588375.0,35845.0,616655.0,0.011102,1.0,0.011102,1.0,4.0,0.0,5,4,0.000016,1,-0.018104
2019-01-01,0,8376.0,406686.0,254.0,68406.0,9.180000,12.0,9.180000,12.0,4.0,0.0,2,6,0.000014,2,-0.018097
2019-01-01,0,686.0,406687.0,20959.0,377791.0,48.290000,12.0,48.290000,12.0,4.0,0.0,1,6,0.000012,1,-0.018069
2019-01-01,0,4019.0,396610.0,6098.0,345197.0,8.740000,4.0,8.740000,4.0,4.0,0.0,3,3,0.000011,1,-0.018098


## Reduce Memory Usage of the DataFrame

In [25]:
import numpy as np

def reduce_mem_usage(props):
    start_mem_usg = props.memory_usage().sum() / 1024**2 
    print(f"Memory usage of dataframe: {start_mem_usg} MB")
    for col in props.columns:
        if props[col].dtype != object:
            
            IsInt = False
            mx = props[col].max()
            mn = props[col].min()

            if not np.isfinite(props[col]).all(): 
                props[col].fillna(mn-1,inplace=True)  
                   
            asint = props[col].fillna(0).astype(np.int64)
            result = (props[col] - asint)
            result = result.sum()
            if result > -0.01 and result < 0.01:
                IsInt = True

            if IsInt:
                if mn >= 0:
                    if mx < 255:
                        props[col] = props[col].astype(np.uint8)
                    elif mx < 65535:
                        props[col] = props[col].astype(np.uint16)
                    elif mx < 4294967295:
                        props[col] = props[col].astype(np.uint32)
                    else:
                        props[col] = props[col].astype(np.uint64)
                else:
                    if mn > np.iinfo(np.int8).min and mx < np.iinfo(np.int8).max:
                        props[col] = props[col].astype(np.int8)
                    elif mn > np.iinfo(np.int16).min and mx < np.iinfo(np.int16).max:
                        props[col] = props[col].astype(np.int16)
                    elif mn > np.iinfo(np.int32).min and mx < np.iinfo(np.int32).max:
                        props[col] = props[col].astype(np.int32)
                    elif mn > np.iinfo(np.int64).min and mx < np.iinfo(np.int64).max:
                        props[col] = props[col].astype(np.int64)    
            
            else:
                props[col] = props[col].astype(np.float32)
    
    mem_usg = props.memory_usage().sum() / 1024**2 
    print(f"MEMORY USAGE AFTER COMPLETION: {mem_usg} MB")
    print(f"{100*mem_usg/start_mem_usg}% of the initial size")
    return props

In [26]:
data = reduce_mem_usage(data)

Memory usage of dataframe: 129.69970703125 MB
MEMORY USAGE AFTER COMPLETION: 44.82269287109375 MB
34.55882352941177% of the initial size


In [27]:
data.to_csv('AMLdata_features_graph.csv')

In [4]:
df = pd.read_csv('AMLdata_features_graph.csv', nrows=2, index_col=0)
df.head()

,Is Laundering,From Bank,Account,To Bank,Account.1,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,Clustering Coeff,In Degree,Out Degree,Degree Centrality,No. of Past Transactions,Z-Score Diff
Timestamp,,,,,,,,,,,,,,,,
2019-01-01,0,10780,187014,10552,187940,164.760000,7,164.760000,7,4,0.0,3,12,0.000027,2,-0.017984
2019-01-01,0,35786,588375,35845,616655,0.011102,1,0.011102,1,4,0.0,5,4,0.000016,1,-0.018104


> upload to box and create shared link, so that you can get the data via wget onto the server

In [30]:
sql = 'CREATE TABLE IF NOT EXISTS public.transactions('
for (a, b) in zip(data.columns, data.dtypes):
    b = "bigint" if "int" in str(b) else "real"
    sql += f' "{a}" {b},'
    
sql = sql[:-1]
sql += ');'
sql

'CREATE TABLE IF NOT EXISTS public.transactions( "Is Laundering" bigint, "From Bank" bigint, "Account" bigint, "To Bank" bigint, "Account.1" bigint, "Amount Received" real, "Receiving Currency" bigint, "Amount Paid" real, "Payment Currency" bigint, "Payment Format" bigint, "Clustering Coeff" real, "In Degree" bigint, "Out Degree" bigint, "Degree Centrality" real, "No. of Past Transactions" bigint, "Z-Score Diff" real);'

```
POSTGRES_POD=$(oc get po -l name=postgresql -o jsonpath="{.items[0].metadata.name}")

oc port-forward ${POSTGRES_POD} 5433:5432

DATA_FILE=AMLdata_features_graph.csv
wget <shared_box_link> -O $DATA_FILE
sed -i 's/\$//g' $DATA_FILE
cat > init-transactions.sql <<EOF
CREATE TABLE IF NOT EXISTS public.laundering_transactions
(
    "Date" date,
    "Is Laundering" bigint,
    "From Bank" bigint,
    "Account" bigint,
    "To Bank" bigint,
    "Account.1" bigint,
    "Amount Received" real,
    "Receiving Currency" bigint,
    "Amount Paid" real,
    "Payment Currency" bigint,
    "Payment Format" bigint,
    "Clustering Coeff" real,
    "In Degree" bigint,
    "Out Degree" bigint,
    "Degree Centrality" real,
    "No. of Past Transactions" bigint,
    "Z-Score Diff" real
);

\copy public.laundering_transactions FROM '$DATA_FILE' WITH (FORMAT csv, HEADER true, DELIMITER ',');
EOF
psql -h 127.0.0.1 -p 5433 -U admin -d stock-prices -a -f init-transactions.sql
```